In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn import preprocessing
min_max_scaler = lambda x: (x-np.min(x))/(np.max(x)-np.min(x))

In [3]:
 pollution_file = 'pollution_2000_2021.csv'

In [82]:
v = pd.read_csv(pollution_file,
               sep=',')
v

,Date,Year,Month,Day,Address,State,County,City,O3 Mean,O3 1st Max Value,...,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI
0,2000-01-01,2000,1,1,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.019765,0.040,...,23,25.0,3.000000,9.0,21,13.0,19.041667,49.0,19,46
1,2000-01-02,2000,1,2,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.015882,0.032,...,0,26.0,1.958333,3.0,22,4.0,22.958333,36.0,19,34
2,2000-01-03,2000,1,3,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.009353,0.016,...,8,28.0,5.250000,11.0,19,16.0,38.125000,51.0,8,48
3,2000-01-04,2000,1,4,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.015882,0.033,...,23,34.0,7.083333,16.0,8,23.0,40.260870,74.0,8,72
4,2000-01-05,2000,1,5,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.007353,0.012,...,2,42.0,8.708333,15.0,7,21.0,48.450000,61.0,22,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608694,2021-06-26,2021,6,26,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,0.040588,0.049,...,0,1.0,0.409091,0.7,2,0.0,1.281818,2.1,0,2
608695,2021-06-27,2021,6,27,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,0.037882,0.042,...,0,1.0,0.325000,0.4,4,0.0,1.104167,2.3,20,2
608696,2021-06-28,2021,6,28,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,0.039765,0.050,...,0,1.0,0.326087,0.5,2,0.0,1.678261,5.8,23,5
608697,2021-06-29,2021,6,29,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,0.049176,0.058,...,0,1.0,0.317391,0.5,20,0.0,2.934783,7.4,0,7


In [5]:
v.dtypes

Date                  object
Year                   int64
Month                  int64
Day                    int64
Address               object
State                 object
County                object
City                  object
O3 Mean              float64
O3 1st Max Value     float64
O3 1st Max Hour        int64
O3 AQI                 int64
CO Mean              float64
CO 1st Max Value     float64
CO 1st Max Hour        int64
CO AQI               float64
SO2 Mean             float64
SO2 1st Max Value    float64
SO2 1st Max Hour       int64
SO2 AQI              float64
NO2 Mean             float64
NO2 1st Max Value    float64
NO2 1st Max Hour       int64
NO2 AQI                int64
dtype: object

In [6]:
la = v[v['City'] == 'Los Angeles']

In [7]:
la = la.copy()

In [8]:
la['Date'] = pd.to_datetime(la['Date'])

In [9]:
la1 = la.groupby(['Date']).mean()

In [10]:
la1

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI
Date,,,,,,,,,,,,,,,,,,,
2000-03-01,2000.0,3.0,1.0,0.011824,0.0270,9.0,25.0,1.004167,1.60,8.0,18.0,0.000000,0.00,0.0,0.0,33.521739,48.00,22.0,45.0
2000-03-02,2000.0,3.0,2.0,0.011235,0.0250,10.0,23.0,1.537500,2.20,4.0,25.0,0.173913,1.00,19.0,1.0,44.173913,55.00,18.0,52.0
2000-03-03,2000.0,3.0,3.0,0.015294,0.0250,10.0,23.0,0.729167,1.00,0.0,11.0,0.391304,3.00,5.0,4.0,32.608696,42.00,18.0,40.0
2000-03-04,2000.0,3.0,4.0,0.021647,0.0290,12.0,27.0,0.712500,1.00,8.0,11.0,0.260870,3.00,5.0,4.0,31.347826,42.00,10.0,40.0
2000-03-05,2000.0,3.0,5.0,0.018882,0.0320,11.0,30.0,0.404167,0.80,23.0,9.0,0.086957,2.00,5.0,3.0,16.956522,42.00,22.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-26,2021.0,6.0,26.0,0.028435,0.0448,9.8,41.2,0.240000,0.30,4.0,3.4,0.553333,0.98,7.2,0.8,11.685834,21.62,18.4,20.4
2021-06-27,2021.0,6.0,27.0,0.034377,0.0532,9.8,55.0,0.263333,0.34,0.8,4.2,0.544167,0.80,4.8,0.8,10.852500,21.80,5.4,20.4
2021-06-28,2021.0,6.0,28.0,0.036717,0.0552,10.2,58.6,0.259167,0.36,7.6,4.4,0.383333,0.76,15.8,0.0,11.669166,21.40,11.0,20.0


In [11]:
la2 = la1[(la1['Year'] > 2009) & (la1['Year'] < 2018)]

In [12]:
la2 = la2.truncate(after='20170909')

In [13]:
crime_file = 'Crime_Data_2010_2017.csv'
j = pd.read_csv(crime_file,
               sep=',')

In [14]:
j['Date Occurred'] = pd.to_datetime(j['Date Occurred'])

In [16]:
count = j.groupby(['Date Occurred']).count()
count

,DR Number,Date Reported,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
Date Occurred,,,,,,,,,,,,,,,,,,,,,
2010-01-01,2135,2135,2135,2135,2135,2135,2135,2132,1985,1863,...,582,2135,2135,2135,124,4,0,2135,146,2135
2010-01-02,533,533,533,533,533,533,533,533,463,426,...,193,533,533,533,50,0,0,533,101,533
2010-01-03,539,539,539,539,539,539,539,539,468,435,...,203,539,539,539,41,0,0,539,102,539
2010-01-04,558,558,558,558,558,558,558,558,493,461,...,171,558,558,558,40,0,0,558,89,558
2010-01-05,546,546,546,546,546,546,546,546,478,446,...,154,546,546,546,39,0,0,546,70,546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05,441,441,441,441,441,441,441,441,392,405,...,176,441,441,441,18,0,0,441,94,441
2017-09-06,361,361,361,361,361,361,361,361,312,334,...,128,361,361,361,24,2,1,361,61,361
2017-09-07,242,242,242,242,242,242,242,242,223,228,...,115,242,242,242,18,1,0,242,46,242


In [17]:
la2.dtypes

Year                 float64
Month                float64
Day                  float64
O3 Mean              float64
O3 1st Max Value     float64
O3 1st Max Hour      float64
O3 AQI               float64
CO Mean              float64
CO 1st Max Value     float64
CO 1st Max Hour      float64
CO AQI               float64
SO2 Mean             float64
SO2 1st Max Value    float64
SO2 1st Max Hour     float64
SO2 AQI              float64
NO2 Mean             float64
NO2 1st Max Value    float64
NO2 1st Max Hour     float64
NO2 AQI              float64
dtype: object

In [18]:
count

,DR Number,Date Reported,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
Date Occurred,,,,,,,,,,,,,,,,,,,,,
2010-01-01,2135,2135,2135,2135,2135,2135,2135,2132,1985,1863,...,582,2135,2135,2135,124,4,0,2135,146,2135
2010-01-02,533,533,533,533,533,533,533,533,463,426,...,193,533,533,533,50,0,0,533,101,533
2010-01-03,539,539,539,539,539,539,539,539,468,435,...,203,539,539,539,41,0,0,539,102,539
2010-01-04,558,558,558,558,558,558,558,558,493,461,...,171,558,558,558,40,0,0,558,89,558
2010-01-05,546,546,546,546,546,546,546,546,478,446,...,154,546,546,546,39,0,0,546,70,546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05,441,441,441,441,441,441,441,441,392,405,...,176,441,441,441,18,0,0,441,94,441
2017-09-06,361,361,361,361,361,361,361,361,312,334,...,128,361,361,361,24,2,1,361,61,361
2017-09-07,242,242,242,242,242,242,242,242,223,228,...,115,242,242,242,18,1,0,242,46,242


In [19]:
crime_count = count['DR Number']

In [20]:
crime_count.mean()

564.0142399430403

In [21]:
c_poll = pd.concat([la2,crime_count],axis=1)
c_poll

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
2010-01-01,2010.0,1.0,1.0,0.017176,0.030000,15.500000,27.500000,0.463889,0.750000,14.500000,8.500000,0.200000,1.000000,10.500000,1.000000,23.152174,40.100000,19.500000,38.000000,2135
2010-01-02,2010.0,1.0,2.0,0.015618,0.025000,9.000000,23.000000,0.502084,0.800000,5.500000,9.000000,0.169565,0.850000,8.500000,0.500000,24.689130,37.850000,11.500000,35.500000,533
2010-01-03,2010.0,1.0,3.0,0.016323,0.030000,9.000000,28.000000,0.427083,0.650000,11.500000,7.500000,0.156522,0.650000,11.000000,0.500000,23.013340,37.000000,19.500000,35.000000,539
2010-01-04,2010.0,1.0,4.0,0.010030,0.020000,9.500000,18.500000,0.535417,0.800000,11.500000,9.000000,0.484783,1.350000,19.000000,1.500000,29.378260,43.450000,19.500000,40.500000,558
2010-01-05,2010.0,1.0,5.0,0.008705,0.018000,10.000000,17.000000,0.597917,1.050000,23.000000,12.000000,0.604348,1.900000,22.000000,2.000000,33.497727,49.700000,20.500000,46.500000,546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05,2017.0,9.0,5.0,0.023922,0.040000,7.666667,37.000000,0.208333,0.266667,8.666667,3.000000,0.125000,0.333333,21.666667,0.000000,8.359343,25.500000,22.333333,23.666667,441
2017-09-06,2017.0,9.0,6.0,0.022132,0.038667,9.666667,35.666667,0.339815,0.500000,5.666667,5.666667,0.249603,0.866667,14.333333,0.333333,16.692460,34.766667,12.333333,32.666667,361
2017-09-07,2017.0,9.0,7.0,0.026882,0.041333,9.666667,38.333333,0.383333,0.600000,6.666667,6.666667,0.200000,0.700000,12.666667,0.333333,16.720833,41.466667,5.333333,38.666667,242
2017-09-08,2017.0,9.0,8.0,0.033157,0.039000,11.333333,36.000000,0.263889,0.333333,4.000000,4.000000,0.165278,0.766667,11.000000,0.000000,11.447222,22.333333,5.000000,20.333333,138


In [22]:
c_poll['DR Number']=c_poll[['DR Number']].apply(min_max_scaler)


In [23]:
c_poll.corr(method='pearson')

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,-0.071970,-0.003232,0.192172,0.231169,-0.039164,0.229707,-0.129464,-0.136498,-0.030192,-0.135016,0.002330,-0.140546,0.111801,-0.196652,-0.113357,-0.119062,-0.022901,-0.120104,0.183706
Month,-0.071970,1.000000,0.010618,-0.062564,-0.016049,-0.064955,0.001703,-0.031001,-0.001334,0.060586,-0.001822,-0.037035,-0.051730,0.045440,-0.064732,-0.007893,-0.045884,0.016210,-0.045618,-0.024146
Day,-0.003232,0.010618,1.000000,0.027550,0.019620,0.031054,0.014081,-0.039557,-0.029146,-0.003722,-0.028110,-0.014422,-0.015884,-0.022175,-0.016228,-0.035882,-0.027994,-0.035535,-0.027083,-0.137944
O3 Mean,0.192172,-0.062564,0.027550,1.000000,0.864605,0.060180,0.770866,-0.524167,-0.558220,-0.291525,-0.559943,-0.349225,-0.224361,-0.180764,-0.202175,-0.637456,-0.487185,-0.357618,-0.484895,0.050181
O3 1st Max Value,0.231169,-0.016049,0.019620,0.864605,1.000000,-0.044133,0.944517,-0.376324,-0.385415,-0.196501,-0.384656,-0.199994,-0.123147,-0.066616,-0.119425,-0.384232,-0.195871,-0.152083,-0.195409,0.082181
O3 1st Max Hour,-0.039164,-0.064955,0.031054,0.060180,-0.044133,1.000000,-0.035498,0.039646,0.041890,-0.058239,0.041690,0.001741,0.057529,-0.068367,0.059321,0.035101,0.048963,-0.125883,0.048774,0.015359
O3 AQI,0.229707,0.001703,0.014081,0.770866,0.944517,-0.035498,1.000000,-0.273311,-0.280196,-0.160551,-0.279495,-0.112368,-0.062771,-0.018929,-0.063473,-0.255641,-0.088986,-0.115384,-0.088422,0.089241
CO Mean,-0.129464,-0.031001,-0.039557,-0.524167,-0.376324,0.039646,-0.273311,1.000000,0.963931,0.095698,0.962790,0.575331,0.421212,0.130719,0.384219,0.815396,0.726092,0.164112,0.725827,-0.072258
CO 1st Max Value,-0.136498,-0.001334,-0.029146,-0.558220,-0.385415,0.041890,-0.280196,0.963931,1.000000,0.154433,0.998573,0.585546,0.443150,0.152018,0.404475,0.846210,0.783923,0.174608,0.782849,-0.065153
CO 1st Max Hour,-0.030192,0.060586,-0.003722,-0.291525,-0.196501,-0.058239,-0.160551,0.095698,0.154433,1.000000,0.157233,0.085074,0.061309,0.196608,0.052204,0.201012,0.225491,0.301612,0.223596,0.014790


In [24]:
c_poll.corr(method='kendall')

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,-0.051567,-0.002404,0.132003,0.160993,-0.003418,0.160883,-0.094305,-0.092197,-0.015832,-0.087451,0.081839,-0.052310,0.090170,-0.132194,-0.075179,-0.084424,-0.017149,-0.085451,0.253623
Month,-0.051567,1.000000,0.007213,-0.039050,-0.000987,-0.089050,-0.001150,-0.034685,-0.017187,0.019411,-0.017269,-0.030193,-0.026449,0.031466,-0.029415,-0.006810,-0.044285,0.016503,-0.044526,0.017286
Day,-0.002404,0.007213,1.000000,0.016774,0.012620,0.005773,0.012930,-0.016827,-0.012248,-0.004840,-0.012611,-0.001560,-0.005438,-0.004808,-0.008751,-0.020082,-0.018380,-0.024416,-0.018044,-0.008832
O3 Mean,0.132003,-0.039050,0.016774,1.000000,0.670188,0.074310,0.668911,-0.341761,-0.374811,-0.128441,-0.376640,-0.226963,-0.196643,-0.107276,-0.192523,-0.440801,-0.351870,-0.240592,-0.352790,0.078288
O3 1st Max Value,0.160993,-0.000987,0.012620,0.670188,1.000000,0.033431,0.987520,-0.198812,-0.213107,-0.062025,-0.214324,-0.110848,-0.094442,-0.039211,-0.093812,-0.237516,-0.147864,-0.107066,-0.148462,0.104517
O3 1st Max Hour,-0.003418,-0.089050,0.005773,0.074310,0.033431,1.000000,0.033555,0.062001,0.057964,0.015896,0.058047,0.015067,0.037123,-0.030906,0.042840,0.075605,0.070738,-0.062182,0.071207,0.015543
O3 AQI,0.160883,-0.001150,0.012930,0.668911,0.987520,0.033555,1.000000,-0.198748,-0.212948,-0.062097,-0.214264,-0.110050,-0.093651,-0.038278,-0.093302,-0.236969,-0.147387,-0.106711,-0.147969,0.105469
CO Mean,-0.094305,-0.034685,-0.016827,-0.341761,-0.198812,0.062001,-0.198748,1.000000,0.859262,0.067668,0.854912,0.399070,0.371304,0.100383,0.366255,0.653012,0.585437,0.129954,0.586638,-0.091125
CO 1st Max Value,-0.092197,-0.017187,-0.012248,-0.374811,-0.213107,0.057964,-0.212948,0.859262,1.000000,0.114407,0.980523,0.416360,0.394208,0.116643,0.389593,0.688654,0.647840,0.136327,0.649255,-0.090830
CO 1st Max Hour,-0.015832,0.019411,-0.004840,-0.128441,-0.062025,0.015896,-0.062097,0.067668,0.114407,1.000000,0.115818,0.070186,0.075139,0.101089,0.068763,0.133258,0.146597,0.164885,0.147053,0.019707


In [25]:
c_poll.corr(method='spearman')

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,-0.069773,-0.003333,0.185852,0.226447,-0.005521,0.225828,-0.132626,-0.127620,-0.023269,-0.120206,0.095208,-0.083259,0.126792,-0.178278,-0.104969,-0.120211,-0.023333,-0.121078,0.363531
Month,-0.069773,1.000000,0.010271,-0.038860,-0.000434,-0.121973,-0.000662,-0.044341,-0.022375,0.027911,-0.023263,-0.042967,-0.038570,0.044667,-0.041891,-0.019874,-0.064107,0.009209,-0.064163,0.025318
Day,-0.003333,0.010271,1.000000,0.024585,0.018691,0.008093,0.019053,-0.024446,-0.017761,-0.007130,-0.018057,-0.002347,-0.008173,-0.007045,-0.011981,-0.029551,-0.027002,-0.035930,-0.026452,-0.015490
O3 Mean,0.185852,-0.038860,0.024585,1.000000,0.853453,0.104918,0.851621,-0.493271,-0.532231,-0.186369,-0.534149,-0.333225,-0.289671,-0.159282,-0.264259,-0.616912,-0.503432,-0.367312,-0.503167,0.116913
O3 1st Max Value,0.226447,-0.000434,0.018691,0.853453,1.000000,0.048675,0.999324,-0.299617,-0.317457,-0.091513,-0.319007,-0.164262,-0.139347,-0.059815,-0.129967,-0.346652,-0.213132,-0.163975,-0.213177,0.154393
O3 1st Max Hour,-0.005521,-0.121973,0.008093,0.104918,0.048675,1.000000,0.048960,0.088642,0.081879,0.022058,0.081520,0.022290,0.053434,-0.042480,0.057244,0.107592,0.102002,-0.089980,0.102430,0.022717
O3 AQI,0.225828,-0.000662,0.019053,0.851621,0.999324,0.048960,1.000000,-0.299301,-0.317073,-0.091592,-0.318740,-0.163040,-0.138103,-0.058232,-0.129195,-0.345471,-0.212242,-0.163324,-0.212276,0.155521
CO Mean,-0.132626,-0.044341,-0.024446,-0.493271,-0.299617,0.088642,-0.299301,1.000000,0.969360,0.099499,0.967611,0.568597,0.531803,0.147021,0.496181,0.835419,0.779820,0.201395,0.779476,-0.135411
CO 1st Max Value,-0.127620,-0.022375,-0.017761,-0.532231,-0.317457,0.081879,-0.317073,0.969360,1.000000,0.162639,0.998119,0.586153,0.557823,0.167743,0.522416,0.865529,0.836439,0.209139,0.836012,-0.133679
CO 1st Max Hour,-0.023269,0.027911,-0.007130,-0.186369,-0.091513,0.022058,-0.091592,0.099499,0.162639,1.000000,0.163775,0.103307,0.109034,0.145517,0.092863,0.190694,0.212209,0.226303,0.212032,0.029247


In [26]:
c_poll[(c_poll['O3 Mean' ]>0.016) & (c_poll['SO2 Mean' ] > 0.7)].corr(method='spearman')

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,0.164034,-0.082648,0.095031,0.244367,-0.129646,0.241483,-0.079193,-0.058396,-0.101418,-0.070985,-0.236851,-0.402019,0.127180,-0.425101,-0.047177,-0.123617,-0.056796,-0.129692,0.276106
Month,0.164034,1.000000,-0.116336,0.147727,0.257165,-0.284178,0.259170,-0.169333,-0.203825,-0.081893,-0.209497,0.063807,-0.107397,0.103604,-0.170432,-0.087334,-0.176926,0.036857,-0.178123,0.120405
Day,-0.082648,-0.116336,1.000000,-0.069401,-0.054265,-0.061977,-0.050886,-0.092519,-0.033567,0.015366,-0.034978,-0.135541,0.041581,0.128980,0.069354,-0.052373,0.030464,-0.021104,0.036180,-0.029600
O3 Mean,0.095031,0.147727,-0.069401,1.000000,0.848574,0.004759,0.846001,-0.465808,-0.475134,0.096602,-0.471044,-0.074278,-0.004270,0.139067,-0.036995,-0.495204,-0.252189,-0.330873,-0.245788,0.306835
O3 1st Max Value,0.244367,0.257165,-0.054265,0.848574,1.000000,-0.099932,0.998249,-0.333359,-0.354146,0.063535,-0.353140,-0.146175,-0.119194,0.146385,-0.128387,-0.254811,-0.049967,-0.094160,-0.046663,0.361498
O3 1st Max Hour,-0.129646,-0.284178,-0.061977,0.004759,-0.099932,1.000000,-0.106406,0.280603,0.268748,-0.043245,0.272414,-0.013719,0.173466,-0.107122,0.200388,0.224903,0.261396,0.058759,0.260608,-0.082454
O3 AQI,0.241483,0.259170,-0.050886,0.846001,0.998249,-0.106406,1.000000,-0.345463,-0.365989,0.071592,-0.365376,-0.142992,-0.119342,0.154526,-0.128308,-0.264879,-0.062326,-0.090947,-0.058834,0.362247
CO Mean,-0.079193,-0.169333,-0.092519,-0.465808,-0.333359,0.280603,-0.345463,1.000000,0.971609,-0.187351,0.971937,0.018709,0.154813,-0.140019,0.164450,0.838130,0.711913,0.274917,0.705574,-0.332247
CO 1st Max Value,-0.058396,-0.203825,-0.033567,-0.475134,-0.354146,0.268748,-0.365989,0.971609,1.000000,-0.136778,0.998072,-0.008826,0.163897,-0.133967,0.179837,0.832102,0.738109,0.251855,0.732171,-0.331308
CO 1st Max Hour,-0.101418,-0.081893,0.015366,0.096602,0.063535,-0.043245,0.071592,-0.187351,-0.136778,1.000000,-0.129834,0.085777,0.076836,-0.007178,0.067105,-0.116486,0.001131,0.042270,0.001909,0.110961


In [27]:
c_poll[c_poll['SO2 Mean' ] > 0.8].corr(method='spearman')

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,0.106980,-0.015619,0.053788,0.116992,-0.171296,0.115176,0.014660,0.003810,-0.057741,0.002035,-0.115841,-0.177538,0.113715,-0.212389,0.036379,-0.021449,-0.006815,-0.024679,-0.039777
Month,0.106980,1.000000,-0.038252,0.004905,0.048787,-0.185666,0.049350,-0.024622,-0.036393,-0.110494,-0.041107,0.099964,-0.133294,0.118778,-0.159565,0.001412,-0.060820,0.035764,-0.057410,0.080224
Day,-0.015619,-0.038252,1.000000,-0.125473,-0.132219,-0.068968,-0.129617,0.015950,0.047911,0.004577,0.045687,-0.045162,0.029766,0.108549,0.046508,0.058622,0.066195,-0.019114,0.066281,-0.044296
O3 Mean,0.053788,0.004905,-0.125473,1.000000,0.954096,0.344111,0.953223,-0.745118,-0.745163,-0.047768,-0.741347,-0.132243,0.011283,-0.099420,-0.040510,-0.783846,-0.535535,-0.356400,-0.533775,0.336411
O3 1st Max Value,0.116992,0.048787,-0.132219,0.954096,1.000000,0.273577,0.999597,-0.727198,-0.731241,-0.041624,-0.728662,-0.126090,-0.007186,-0.076124,-0.051976,-0.722436,-0.465976,-0.251874,-0.464746,0.350345
O3 1st Max Hour,-0.171296,-0.185666,-0.068968,0.344111,0.273577,1.000000,0.274043,-0.202125,-0.207547,-0.000452,-0.207148,-0.090005,0.212991,-0.175772,0.220695,-0.254593,-0.092112,-0.093621,-0.087182,0.082647
O3 AQI,0.115176,0.049350,-0.129617,0.953223,0.999597,0.274043,1.000000,-0.728452,-0.731725,-0.038255,-0.729257,-0.126297,-0.006130,-0.076517,-0.051014,-0.723060,-0.467157,-0.250201,-0.465790,0.350569
CO Mean,0.014660,-0.024622,0.015950,-0.745118,-0.727198,-0.202125,-0.728452,1.000000,0.963467,-0.045394,0.964033,0.199166,0.066624,0.010684,0.102590,0.846973,0.718779,0.231607,0.716648,-0.401160
CO 1st Max Value,0.003810,-0.036393,0.047911,-0.745163,-0.731241,-0.207547,-0.731725,0.963467,1.000000,-0.010223,0.999004,0.185317,0.114030,0.047382,0.151260,0.829643,0.738536,0.209250,0.735452,-0.384870
CO 1st Max Hour,-0.057741,-0.110494,0.004577,-0.047768,-0.041624,-0.000452,-0.038255,-0.045394,-0.010223,1.000000,-0.008102,0.095232,0.030823,0.072796,0.010120,0.011493,0.106414,0.080287,0.102956,0.095209


In [28]:
c_poll1=c_poll.rename(index = str, columns={"DR Number":"total_crime","O3 Mean" : "O3_Mean","CO Mean":"CO_Mean","SO2 Mean":"SO2_Mean","NO2 Mean":"NO2_Mean"})

In [31]:
c_poll1
test = c_poll1[(c_poll1['O3_Mean' ]>0.016) & (c_poll1['SO2_Mean' ] > 0.7)]

In [32]:
crime_model = ols('total_crime ~ CO_Mean+O3_Mean+SO2_Mean+NO2_Mean', data=c_poll1).fit()


In [208]:
crime_summary = crime_model.summary()
crime_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_crime   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     22.72
Date:                Mon, 02 May 2022   Prob (F-statistic):           1.76e-18
Time:                        00:26:29   Log-Likelihood:                 4697.9
No. Observations:                2665   AIC:                            -9386.
Df Residuals:                    2660   BIC:                            -9356.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2524      0.005     55.047      0.000       0.243       0.261
CO_Mean       -0.0358      0.006     -6.051      0.000      -0.047      -0.024
O3_Mean        0.1938      0.107      1.817      0.069      -0.015       0.403
SO2_Mean       0.0267      0.003      7.764      0.000       0.020       0.033
NO2_Mean       0.0002      0.000      1.409      0.159   -9.45e-05       0.001
==============================================================================
Omnibus:                     3260.099   Durbin-Watson:                   1.596
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           883499.714
Skew:                           6.192   Prob(JB):                         0.00
Kurtosis:                      91.335   Cond. No.                     2.69e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.69e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [209]:
results = model.fit()
print(results.params)

AttributeError: 'OLSResults' object has no attribute 'fit'

In [62]:
crime_model1 = ols('total_crime ~ CO_Mean+O3_Mean+SO2_Mean+NO2_Mean', data=test).fit()
crime_summary1 = crime_model1.summary()
crime_summary1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_crime   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     4.827
Date:                Mon, 02 May 2022   Prob (F-statistic):           0.000986
Time:                        20:15:45   Log-Likelihood:                 391.89
No. Observations:                 198   AIC:                            -773.8
Df Residuals:                     193   BIC:                            -757.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2742      0.017     16.313      0.000       0.241       0.307
CO_Mean       -0.0347      0.013     -2.669      0.008      -0.060      -0.009
O3_Mean        0.5069      0.333      1.521      0.130      -0.150       1.164
SO2_Mean      -0.0120      0.008     -1.424      0.156      -0.029       0.005
NO2_Mean       0.0003      0.000      0.728      0.467      -0.000       0.001
==============================================================================
Omnibus:                      100.764   Durbin-Watson:                   1.852
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              555.964
Skew:                           1.913   Prob(JB):                    1.88e-121
Kurtosis:                      10.263   Cond. No.                     3.59e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.59e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [68]:
j.groupby(['Weapon Description']).count().sort_values('DR Number',ascending=False)


,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Used Code,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
Weapon Description,,,,,,,,,,,,,,,,,,,,,
"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",319818,319818,319818,319818,319818,319818,319818,319818,319716,316845,...,319818,319818,319818,319817,19564,702,27,319818,78061,319816
VERBAL THREAT,43814,43814,43814,43814,43814,43814,43814,43814,43783,42710,...,43814,43814,43814,43813,2627,113,4,43814,5750,43814
UNKNOWN WEAPON/OTHER WEAPON,40746,40746,40746,40746,40746,40746,40746,40746,40739,40190,...,40746,40746,40746,40746,4133,67,2,40746,8275,40746
HAND GUN,25352,25352,25352,25352,25352,25352,25352,25352,25345,24999,...,25352,25352,25352,25351,5848,176,5,25352,10228,25352
SEMI-AUTOMATIC PISTOL,10096,10096,10096,10096,10096,10096,10096,10096,10096,10004,...,10096,10096,10096,10096,2377,59,3,10096,3936,10096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAC-10 SEMIAUTOMATIC ASSAULT WEAPON,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,2,0,0,8,5,8
ANTIQUE FIREARM,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,0,0,0,7,2,7
MAC-11 SEMIAUTOMATIC ASSAULT WEAPON,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,1,0,0,5,2,5


In [72]:
fist = j[j['Weapon Description'] == 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)']
f_count = fist.groupby(['Date Occurred']).count()
f_crime_count = f_count['DR Number']
f_c_poll = pd.concat([la2,f_crime_count],axis=1)
f_c_poll['DR Number']=f_c_poll[['DR Number']].apply(min_max_scaler)

In [73]:
f_count

,DR Number,Date Reported,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
Date Occurred,,,,,,,,,,,,,,,,,,,,,
2010-01-01,453,453,453,453,453,453,453,450,438,346,...,453,453,453,453,76,2,0,453,62,453
2010-01-02,123,123,123,123,123,123,123,123,120,118,...,123,123,123,123,8,0,0,123,40,123
2010-01-03,123,123,123,123,123,123,123,123,120,120,...,123,123,123,123,6,0,0,123,32,123
2010-01-04,100,100,100,100,100,100,100,100,99,95,...,100,100,100,100,7,0,0,100,29,100
2010-01-05,87,87,87,87,87,87,87,87,87,83,...,87,87,87,87,5,0,0,87,16,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05,99,99,99,99,99,99,99,99,99,96,...,99,99,99,99,6,0,0,99,25,99
2017-09-06,85,85,85,85,85,85,85,85,85,83,...,85,85,85,85,7,1,1,85,11,85
2017-09-07,68,68,68,68,68,68,68,68,67,68,...,68,68,68,68,1,0,0,68,14,68


In [74]:
f_c_poll.corr(method='pearson')

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,-0.071970,-0.003232,0.192172,0.231169,-0.039164,0.229707,-0.129464,-0.136498,-0.030192,-0.135016,0.002330,-0.140546,0.111801,-0.196652,-0.113357,-0.119062,-0.022901,-0.120104,0.076221
Month,-0.071970,1.000000,0.010618,-0.062564,-0.016049,-0.064955,0.001703,-0.031001,-0.001334,0.060586,-0.001822,-0.037035,-0.051730,0.045440,-0.064732,-0.007893,-0.045884,0.016210,-0.045618,0.007180
Day,-0.003232,0.010618,1.000000,0.027550,0.019620,0.031054,0.014081,-0.039557,-0.029146,-0.003722,-0.028110,-0.014422,-0.015884,-0.022175,-0.016228,-0.035882,-0.027994,-0.035535,-0.027083,-0.083292
O3 Mean,0.192172,-0.062564,0.027550,1.000000,0.864605,0.060180,0.770866,-0.524167,-0.558220,-0.291525,-0.559943,-0.349225,-0.224361,-0.180764,-0.202175,-0.637456,-0.487185,-0.357618,-0.484895,0.232590
O3 1st Max Value,0.231169,-0.016049,0.019620,0.864605,1.000000,-0.044133,0.944517,-0.376324,-0.385415,-0.196501,-0.384656,-0.199994,-0.123147,-0.066616,-0.119425,-0.384232,-0.195871,-0.152083,-0.195409,0.293214
O3 1st Max Hour,-0.039164,-0.064955,0.031054,0.060180,-0.044133,1.000000,-0.035498,0.039646,0.041890,-0.058239,0.041690,0.001741,0.057529,-0.068367,0.059321,0.035101,0.048963,-0.125883,0.048774,-0.089911
O3 AQI,0.229707,0.001703,0.014081,0.770866,0.944517,-0.035498,1.000000,-0.273311,-0.280196,-0.160551,-0.279495,-0.112368,-0.062771,-0.018929,-0.063473,-0.255641,-0.088986,-0.115384,-0.088422,0.281204
CO Mean,-0.129464,-0.031001,-0.039557,-0.524167,-0.376324,0.039646,-0.273311,1.000000,0.963931,0.095698,0.962790,0.575331,0.421212,0.130719,0.384219,0.815396,0.726092,0.164112,0.725827,-0.139724
CO 1st Max Value,-0.136498,-0.001334,-0.029146,-0.558220,-0.385415,0.041890,-0.280196,0.963931,1.000000,0.154433,0.998573,0.585546,0.443150,0.152018,0.404475,0.846210,0.783923,0.174608,0.782849,-0.131444
CO 1st Max Hour,-0.030192,0.060586,-0.003722,-0.291525,-0.196501,-0.058239,-0.160551,0.095698,0.154433,1.000000,0.157233,0.085074,0.061309,0.196608,0.052204,0.201012,0.225491,0.301612,0.223596,-0.128417


In [75]:
f_c_poll.corr(method='spearman')

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,-0.069773,-0.003333,0.185852,0.226447,-0.005521,0.225828,-0.132626,-0.127620,-0.023269,-0.120206,0.095208,-0.083259,0.126792,-0.178278,-0.104969,-0.120211,-0.023333,-0.121078,0.124960
Month,-0.069773,1.000000,0.010271,-0.038860,-0.000434,-0.121973,-0.000662,-0.044341,-0.022375,0.027911,-0.023263,-0.042967,-0.038570,0.044667,-0.041891,-0.019874,-0.064107,0.009209,-0.064163,0.048140
Day,-0.003333,0.010271,1.000000,0.024585,0.018691,0.008093,0.019053,-0.024446,-0.017761,-0.007130,-0.018057,-0.002347,-0.008173,-0.007045,-0.011981,-0.029551,-0.027002,-0.035930,-0.026452,-0.025284
O3 Mean,0.185852,-0.038860,0.024585,1.000000,0.853453,0.104918,0.851621,-0.493271,-0.532231,-0.186369,-0.534149,-0.333225,-0.289671,-0.159282,-0.264259,-0.616912,-0.503432,-0.367312,-0.503167,0.302996
O3 1st Max Value,0.226447,-0.000434,0.018691,0.853453,1.000000,0.048675,0.999324,-0.299617,-0.317457,-0.091513,-0.319007,-0.164262,-0.139347,-0.059815,-0.129967,-0.346652,-0.213132,-0.163975,-0.213177,0.366476
O3 1st Max Hour,-0.005521,-0.121973,0.008093,0.104918,0.048675,1.000000,0.048960,0.088642,0.081879,0.022058,0.081520,0.022290,0.053434,-0.042480,0.057244,0.107592,0.102002,-0.089980,0.102430,-0.068532
O3 AQI,0.225828,-0.000662,0.019053,0.851621,0.999324,0.048960,1.000000,-0.299301,-0.317073,-0.091592,-0.318740,-0.163040,-0.138103,-0.058232,-0.129195,-0.345471,-0.212242,-0.163324,-0.212276,0.367333
CO Mean,-0.132626,-0.044341,-0.024446,-0.493271,-0.299617,0.088642,-0.299301,1.000000,0.969360,0.099499,0.967611,0.568597,0.531803,0.147021,0.496181,0.835419,0.779820,0.201395,0.779476,-0.167065
CO 1st Max Value,-0.127620,-0.022375,-0.017761,-0.532231,-0.317457,0.081879,-0.317073,0.969360,1.000000,0.162639,0.998119,0.586153,0.557823,0.167743,0.522416,0.865529,0.836439,0.209139,0.836012,-0.177018
CO 1st Max Hour,-0.023269,0.027911,-0.007130,-0.186369,-0.091513,0.022058,-0.091592,0.099499,0.162639,1.000000,0.163775,0.103307,0.109034,0.145517,0.092863,0.190694,0.212209,0.226303,0.212032,-0.155023


In [76]:
f_c_poll[(f_c_poll['O3 Mean' ]>0.0015) & (f_c_poll['SO2 Mean' ] > 0.8)].corr(method='spearman')

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,0.106980,-0.015619,0.053788,0.116992,-0.171296,0.115176,0.014660,0.003810,-0.057741,0.002035,-0.115841,-0.177538,0.113715,-0.212389,0.036379,-0.021449,-0.006815,-0.024679,-0.050879
Month,0.106980,1.000000,-0.038252,0.004905,0.048787,-0.185666,0.049350,-0.024622,-0.036393,-0.110494,-0.041107,0.099964,-0.133294,0.118778,-0.159565,0.001412,-0.060820,0.035764,-0.057410,0.067737
Day,-0.015619,-0.038252,1.000000,-0.125473,-0.132219,-0.068968,-0.129617,0.015950,0.047911,0.004577,0.045687,-0.045162,0.029766,0.108549,0.046508,0.058622,0.066195,-0.019114,0.066281,-0.049741
O3 Mean,0.053788,0.004905,-0.125473,1.000000,0.954096,0.344111,0.953223,-0.745118,-0.745163,-0.047768,-0.741347,-0.132243,0.011283,-0.099420,-0.040510,-0.783846,-0.535535,-0.356400,-0.533775,0.532549
O3 1st Max Value,0.116992,0.048787,-0.132219,0.954096,1.000000,0.273577,0.999597,-0.727198,-0.731241,-0.041624,-0.728662,-0.126090,-0.007186,-0.076124,-0.051976,-0.722436,-0.465976,-0.251874,-0.464746,0.569588
O3 1st Max Hour,-0.171296,-0.185666,-0.068968,0.344111,0.273577,1.000000,0.274043,-0.202125,-0.207547,-0.000452,-0.207148,-0.090005,0.212991,-0.175772,0.220695,-0.254593,-0.092112,-0.093621,-0.087182,0.137573
O3 AQI,0.115176,0.049350,-0.129617,0.953223,0.999597,0.274043,1.000000,-0.728452,-0.731725,-0.038255,-0.729257,-0.126297,-0.006130,-0.076517,-0.051014,-0.723060,-0.467157,-0.250201,-0.465790,0.569641
CO Mean,0.014660,-0.024622,0.015950,-0.745118,-0.727198,-0.202125,-0.728452,1.000000,0.963467,-0.045394,0.964033,0.199166,0.066624,0.010684,0.102590,0.846973,0.718779,0.231607,0.716648,-0.506788
CO 1st Max Value,0.003810,-0.036393,0.047911,-0.745163,-0.731241,-0.207547,-0.731725,0.963467,1.000000,-0.010223,0.999004,0.185317,0.114030,0.047382,0.151260,0.829643,0.738536,0.209250,0.735452,-0.500330
CO 1st Max Hour,-0.057741,-0.110494,0.004577,-0.047768,-0.041624,-0.000452,-0.038255,-0.045394,-0.010223,1.000000,-0.008102,0.095232,0.030823,0.072796,0.010120,0.011493,0.106414,0.080287,0.102956,0.005707


In [77]:
f_c_poll1=f_c_poll.rename(index = str, columns={"DR Number":"total_crime","O3 Mean" : "O3_Mean","CO Mean":"CO_Mean","SO2 Mean":"SO2_Mean","NO2 Mean":"NO2_Mean"})

In [78]:
c_poll1

,Year,Month,Day,O3_Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO_Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2_Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2_Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,total_crime
2010-01-01 00:00:00,2010.0,1.0,1.0,0.017176,0.030000,15.500000,27.500000,0.463889,0.750000,14.500000,8.500000,0.200000,1.000000,10.500000,1.000000,23.152174,40.100000,19.500000,38.000000,1.000000
2010-01-02 00:00:00,2010.0,1.0,2.0,0.015618,0.025000,9.000000,23.000000,0.502084,0.800000,5.500000,9.000000,0.169565,0.850000,8.500000,0.500000,24.689130,37.850000,11.500000,35.500000,0.240758
2010-01-03 00:00:00,2010.0,1.0,3.0,0.016323,0.030000,9.000000,28.000000,0.427083,0.650000,11.500000,7.500000,0.156522,0.650000,11.000000,0.500000,23.013340,37.000000,19.500000,35.000000,0.243602
2010-01-04 00:00:00,2010.0,1.0,4.0,0.010030,0.020000,9.500000,18.500000,0.535417,0.800000,11.500000,9.000000,0.484783,1.350000,19.000000,1.500000,29.378260,43.450000,19.500000,40.500000,0.252607
2010-01-05 00:00:00,2010.0,1.0,5.0,0.008705,0.018000,10.000000,17.000000,0.597917,1.050000,23.000000,12.000000,0.604348,1.900000,22.000000,2.000000,33.497727,49.700000,20.500000,46.500000,0.246919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05 00:00:00,2017.0,9.0,5.0,0.023922,0.040000,7.666667,37.000000,0.208333,0.266667,8.666667,3.000000,0.125000,0.333333,21.666667,0.000000,8.359343,25.500000,22.333333,23.666667,0.197156
2017-09-06 00:00:00,2017.0,9.0,6.0,0.022132,0.038667,9.666667,35.666667,0.339815,0.500000,5.666667,5.666667,0.249603,0.866667,14.333333,0.333333,16.692460,34.766667,12.333333,32.666667,0.159242
2017-09-07 00:00:00,2017.0,9.0,7.0,0.026882,0.041333,9.666667,38.333333,0.383333,0.600000,6.666667,6.666667,0.200000,0.700000,12.666667,0.333333,16.720833,41.466667,5.333333,38.666667,0.102844
2017-09-08 00:00:00,2017.0,9.0,8.0,0.033157,0.039000,11.333333,36.000000,0.263889,0.333333,4.000000,4.000000,0.165278,0.766667,11.000000,0.000000,11.447222,22.333333,5.000000,20.333333,0.053555


In [79]:
f_c_poll1

,Year,Month,Day,O3_Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO_Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2_Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2_Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,total_crime
2010-01-01 00:00:00,2010.0,1.0,1.0,0.017176,0.030000,15.500000,27.500000,0.463889,0.750000,14.500000,8.500000,0.200000,1.000000,10.500000,1.000000,23.152174,40.100000,19.500000,38.000000,1.000000
2010-01-02 00:00:00,2010.0,1.0,2.0,0.015618,0.025000,9.000000,23.000000,0.502084,0.800000,5.500000,9.000000,0.169565,0.850000,8.500000,0.500000,24.689130,37.850000,11.500000,35.500000,0.251701
2010-01-03 00:00:00,2010.0,1.0,3.0,0.016323,0.030000,9.000000,28.000000,0.427083,0.650000,11.500000,7.500000,0.156522,0.650000,11.000000,0.500000,23.013340,37.000000,19.500000,35.000000,0.251701
2010-01-04 00:00:00,2010.0,1.0,4.0,0.010030,0.020000,9.500000,18.500000,0.535417,0.800000,11.500000,9.000000,0.484783,1.350000,19.000000,1.500000,29.378260,43.450000,19.500000,40.500000,0.199546
2010-01-05 00:00:00,2010.0,1.0,5.0,0.008705,0.018000,10.000000,17.000000,0.597917,1.050000,23.000000,12.000000,0.604348,1.900000,22.000000,2.000000,33.497727,49.700000,20.500000,46.500000,0.170068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05 00:00:00,2017.0,9.0,5.0,0.023922,0.040000,7.666667,37.000000,0.208333,0.266667,8.666667,3.000000,0.125000,0.333333,21.666667,0.000000,8.359343,25.500000,22.333333,23.666667,0.197279
2017-09-06 00:00:00,2017.0,9.0,6.0,0.022132,0.038667,9.666667,35.666667,0.339815,0.500000,5.666667,5.666667,0.249603,0.866667,14.333333,0.333333,16.692460,34.766667,12.333333,32.666667,0.165533
2017-09-07 00:00:00,2017.0,9.0,7.0,0.026882,0.041333,9.666667,38.333333,0.383333,0.600000,6.666667,6.666667,0.200000,0.700000,12.666667,0.333333,16.720833,41.466667,5.333333,38.666667,0.126984
2017-09-08 00:00:00,2017.0,9.0,8.0,0.033157,0.039000,11.333333,36.000000,0.263889,0.333333,4.000000,4.000000,0.165278,0.766667,11.000000,0.000000,11.447222,22.333333,5.000000,20.333333,0.043084


In [80]:
f_crime_model = ols('total_crime ~ CO_Mean+O3_Mean+SO2_Mean+NO2_Mean', data=f_c_poll1).fit()
f_crime_summary = f_crime_model.summary()
f_crime_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_crime   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     50.19
Date:                Mon, 02 May 2022   Prob (F-statistic):           8.81e-41
Time:                        20:33:35   Log-Likelihood:                 4174.9
No. Observations:                2665   AIC:                            -8340.
Df Residuals:                    2660   BIC:                            -8310.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2045      0.006     36.656      0.000       0.194       0.215
CO_Mean       -0.0125      0.007     -1.734      0.083      -0.027       0.002
O3_Mean        1.1012      0.130      8.484      0.000       0.847       1.356
SO2_Mean       0.0280      0.004      6.688      0.000       0.020       0.036
NO2_Mean      -0.0005      0.000     -2.202      0.028      -0.001   -5.03e-05
==============================================================================
Omnibus:                     2278.937   Durbin-Watson:                   1.513
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           180277.531
Skew:                           3.594   Prob(JB):                         0.00
Kurtosis:                      42.647   Cond. No.                     2.69e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.69e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [81]:
test1 = f_c_poll1[(f_c_poll1['O3_Mean' ]>0.016) & (f_c_poll1['SO2_Mean' ] > 0.7)]
f_crime_model1 = ols('total_crime ~ CO_Mean+O3_Mean+SO2_Mean+NO2_Mean', data=test1).fit()
f_crime_summary1 = f_crime_model1.summary()
f_crime_summary1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_crime   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     6.231
Date:                Mon, 02 May 2022   Prob (F-statistic):           9.77e-05
Time:                        21:49:22   Log-Likelihood:                 346.18
No. Observations:                 198   AIC:                            -682.4
Df Residuals:                     193   BIC:                            -665.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2676      0.021     12.635      0.000       0.226       0.309
CO_Mean       -0.0438      0.016     -2.672      0.008      -0.076      -0.011
O3_Mean        0.5894      0.420      1.404      0.162      -0.239       1.417
SO2_Mean      -0.0121      0.011     -1.137      0.257      -0.033       0.009
NO2_Mean   -8.513e-05      0.000     -0.170      0.865      -0.001       0.001
==============================================================================
Omnibus:                       43.251   Durbin-Watson:                   1.913
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               82.274
Skew:                           1.067   Prob(JB):                     1.36e-18
Kurtosis:                       5.327   Cond. No.                     3.59e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.59e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""